20240306

SarahFong

Take output from DiffBind deseq2 formatting

          $HOME/EMF/US/bin/ATAC/1_prep_deseq2_ATAC.ipynb 
          
Prepare data: 
1. Clean diffbind deseq2 conc data (remove bad coordinates)
4. Format dataframe for LegNet, multitask, deepstarr multitask
5. Split into training and test set - randomly hold out 2 chromosomes. 
6. Add fold num to training set

In [1]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
import config_readwrite as crw
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import pandas as pd
import pybedtools as pbt

from scipy import stats

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import plot_params as pp
pp.fonts()

('sans-serif', 'Arial', 18)

In [2]:
config_name =os.path.join(os.getcwd(), "config.multi.ini")
config, cfn = crw.read(config_name)

# functions

In [3]:
def setPrefixRules(prefix):
    """ rules for processing datasets"""
    if "class.all" in prefix:
        QUANTILE_FILTER=False
        MIN_CONC_FILTER=False
        JOINT_ACCESSIBLE=True
        CLASS_LABEL = True

    elif "class.nojoint" in prefix:
        QUANTILE_FILTER=False
        MIN_CONC_FILTER=True
        JOINT_ACCESSIBLE=False
        CLASS_LABEL = True

    elif "reg.all" in PREFIX:
        QUANTILE_FILTER=False
        MIN_CONC_FILTER=True
        JOINT_ACCESSIBLE=True
        CLASS_LABEL = False

    elif "reg.nojoint" in prefix:
        QUANTILE_FILTER=False
        MIN_CONC_FILTER=True
        JOINT_ACCESSIBLE=False
        CLASS_LABEL = False
    else:
        print('need to add rules for', prefix)
        
    return QUANTILE_FILTER, MIN_CONC_FILTER, JOINT_ACCESSIBLE, CLASS_LABEL


## make chromosome list

In [4]:
def chrList():
    """return  list of chromosomes"""
    
    chrs = []
    
    for n in np.arange(1,23):
        chrs.append(f"chr{n}")
    
    # add sex chromosomes
    chrs.append("chrX")
    chrs.append("chrY")
    
    return chrs

## write fa

In [5]:
def writeFa(heldout_df, heldout_fa):
    with open(heldout_fa, "w") as writer:
        for row in heldout_df.iterrows():
            seqid, seq=row[1][:2]
            writer.write(f">{seqid}\n{seq}\n")

## train test split on chromosome

In [6]:
def splitTrainTestVal(df, val_chr_list=None, test_chr_list=None):
    """randomly sample and hold out 2 chromosomes for testing, validation"""
    
    cols =['coor.type',"seq"]
    
    # randomly sample test chromosomes (n=2)
    if test_chr_list is None:
        print('randomly sampling chromosomes for test')


        # randomly sample
        test_chr_list = list(np.random.choice(chrs, 2))
   
    # randomly sample validation chromosome (n=1)
    if val_chr_list is None:
        print('randomly sampling chromosomes for val')

        # remove test chromosomes
        for chr_ in test_chr_list:
            chrs.remove(chr_)

        # randomly sample
        val_chr_list = list(np.random.choice(chrs, 1))

    # separate held out chromosomes from  training chromosomes
    test = df.loc[df["#chr"].isin(test_chr_list)].copy()
    val = df.loc[df["#chr"].isin(val_chr_list)].copy()
    train = df.loc[(~df["#chr"].isin(test_chr_list))&
                  (~df["#chr"].isin(val_chr_list))].copy()

    return train, val, test


## filters

### need a deseq2 filter. 

In [7]:
def minReadDepthFilter(table, MIN_CONC):

    print("before read depth filter:", table.shape)
    table = table.loc[(table["ctrl"] > MIN_CONC) |
                      (table["US"] > MIN_CONC)].copy()
    print("after:", table.shape)

    return table


def quantileFilter(table, quantile=0.99):
    """upper quantile filter for read count values"""

    print("before quantile filter:", table.shape)

    # quantiles
    ctrl_thresh, us_thresh = table[["ctrl", "US"]].quantile(quantile)

    # filter table
    table = table.loc[(table["ctrl"] < ctrl_thresh) &
                      (table["US"] < us_thresh)]

    print("after:", table.shape)

    return table


def jointAccessibleFilter(table):
    """remove joint accessible regions, scramble new dataframe"""

    print("before joint_accessible filter:", table.shape)

    # filter table
    ctrl_only = table.loc[(table["ctrl"] > 0) &
                          (table["US"] == 0)].copy()

    US_only = table.loc[(table["ctrl"] == 0) &
                        (table["US"] > 0)].copy()

    # combine us and ctrl
    # shuffle dataframe
    table = pd.concat([ctrl_only, US_only]).sample(
        frac=1).reset_index(drop=True)

    print("after:", table.shape)

    return table

# binarize


def classLabel(table):
    """binarize read count column"""
    cols_to_label = ['ctrl', 'US']
    for col in cols_to_label:
        table[col] = table[col].apply(lambda x: 0 if x<MIN_CONC else 1)

    return table

# label deseq DiffAcc peaks
def diffAccLabel(table, diff_bed):
    """ Annotate elements that overlap desseq2 differentially accessible peak calls
        Return dataframe of overlapping elements. 
    """
    table_bed = pbt.BedTool.from_dataframe(table[table.columns[:4]])
    deseq_bed = pbt.BedTool(diff_bed)

    table_deseq = table_bed.intersect(deseq_bed, wao=True)

    table_deseq = pd.read_csv(table_deseq.fn, sep='\t', header=None)
    table_deseq.columns=['#chr', "start_trim", "end_trim", "type", 
                         "#chr_deseq", "start_deseq", "end_deseq", 
                         "deseq_id", "diffAcc"
                        ]

    #annotate diffAcc overlap as 1 or 0
    table_deseq["diffAcc"] = table_deseq["diffAcc"].apply(lambda x: 1 if x>0 else 0) 

    # merge annotations with original table
    table = pd.merge(table, table_deseq[["type","diffAcc"]]).drop_duplicates()
    
    return table

# scramble df


def dfShuffle(df):
    """scramble dataframe"""
    return df.sample(frac=1).reset_index(drop=True)

def directionFilter(table):
    """filter for positive directional changes in activity"""

    table = table.loc[table["US_DIF"]<0] #where US_DIF = ctrl - US, and US_dif <0 means US has more activity than control
    
    return table

# Main

## load data

In [15]:
CL = 'hob'
PREFIX = "class.all.2tasks"

PATH = "/wynton/group/ahituv/data/US-MPRA/ATAC-seq/Diffbind_results"
DATA_PATH = f"/wynton/home/ahituv/fongsl/EMF/US/ml_emf/data/deepstarr/deseq2/{CL}/{PREFIX}"

TEST_CHR = ["chr8"]
VAL_CHR = "chr12"


# peak information
PEAK_SIZE = 270
MIN_CONC = 3
DIRECTION_FILTER = False  # keep only positive changes? Negative changes? 

# genome information
HG38= "/wynton/group/ahituv/data/dna/hg38/hg38.chrom.sizes"
FA_HG38="/wynton/group/ahituv/data/dna/hg38/hg38.fa"


if "class" in PREFIX:
    cols=['coor.type', "seq", "diffAcc", "fold_num"]
else:
    cols = ['coor.type', "seq", "ctrl", "US", "US_DIF", "fold_num"]
    
fa_cols = ['coor.type', "seq"]


if os.path.exists(DATA_PATH) is False:
    if os.path.dirname(DATA_PATH) is False:
        os.mkdir(os.path.dirname(DATA_PATH))
    os.mkdir(DATA_PATH)

## base files

# files

# DESEQ2 information
DIFF = f'./{CL}_deseq2.csv'
NODIFF = f'./{CL}_deseq2-nodiff.csv'
DIFF_BED = "./" + DIFF.strip(".csv") + ".bed"

# write training, test files
FULL = f"{CL}_deseq2-nondiff.trimmed.full.csv"


## base config

section = f"{CL}-ATAC-DESEQ2"
crw.check(config, section)

config[section]["path"] = PATH

config[section]["nondiff_bind_results"] = "%(path)s/" + NODIFF
config[section]["diff_bind_results"] = "%(path)s/" + DIFF
config[section]["diff_bind_results_bed"] = "%(path)s/" + DIFF_BED


## deepstarr files

# file inputs for training, testing
## x
TRAIN_FA = f"{PREFIX}.Sequences_Train.fa"
VAL_FA = f"{PREFIX}.Sequences_Val.fa"
TEST_FA = f"{PREFIX}.Sequences_Test.fa"

## y
TRAIN_TARGET = f"{PREFIX}.Sequences_activity_Train.txt"
VAL_TARGET = f"{PREFIX}.Sequences_activity_Val.txt"
TEST_TARGET = f"{PREFIX}.Sequences_activity_Test.txt"

## rules

## deepstarr config 

section = f"{CL}.atac.deseq2.deepstarr"
crw.check(config, section)

# deepstar config
config[section]["data_path"] = DATA_PATH
config[section]["held_out_chr"] = ",".join(TEST_CHR)
config[section]["val_chr"] = VAL_CHR

## deepstarr+prefix config

# rules for data
QUANTILE_FILTER, MIN_CONC_FILTER, JOINT_ACCESSIBLE, CLASS_LABEL = setPrefixRules(PREFIX)


# deepstarr prefix config
section = f"{CL}.atac.deseq2.deepstarr.{PREFIX}"
crw.check(config, section)

config[section]["train_fa"] = TRAIN_FA
config[section]["val_fa"] = VAL_FA
config[section]["test_fa"] = TEST_FA

config[section]["train_target"] = TRAIN_TARGET
config[section]["tval_target"] = VAL_TARGET
config[section]["test_target"] = TEST_TARGET


config[section]["filter_MIN_CONC"] = str(MIN_CONC_FILTER)
if MIN_CONC_FILTER is True:
    config[section]["MIN_CONC"] = str(MIN_CONC)
    
config[section]["filter_quantile"] = str(QUANTILE_FILTER)
config[section]["filter_jointaccessible"] = str(JOINT_ACCESSIBLE)
config[section]["classlabel"] = str(CLASS_LABEL)

crw.write(config, cfn)

In [10]:
os.chdir(PATH)
table = pd.read_csv(FULL, sep='\t').drop_duplicates()

table["coor.type"] = table["type"] + "|" + table["seq.id"]

if "Conc_Ultrasound_dif" not in list(table):
    table["Conc_Ultrasound_dif"] = table["Conc_Control"] - \
        table["Conc_Ultrasound"]

table.rename(columns={"Conc_Ultrasound": "US",
                      "Conc_Control": "ctrl",
                      "Conc_Ultrasound_dif": "US_DIF"}, inplace=True)


table.head()

,#chr,start_trim,end_trim,type,seq.id,US,ctrl,US_DIF,seq,coor.type
0,chr20,35771828,35772099,hob.1,chr20:35771828-35772099,8.646043,8.194171,-0.451872,TAGGGCGCGGGCCTGTTTCCCGCGCGTCAGGGTAGTGGGCGTGGCC...,hob.1|chr20:35771828-35772099
1,chr3,127075008,127075279,hob.2,chr3:127075008-127075279,7.579306,6.978627,-0.600679,GTGAGCTGGGCTATGCTTGGTTGGTCTGGGCTGAGCTCCACGGTCT...,hob.2|chr3:127075008-127075279
2,chr16,82016933,82017204,hob.3,chr16:82016933-82017204,4.529926,3.056418,-1.473507,CTAGAAACATAGCCTTTAGAGACAGAGTGATGGAGATTTGGATCCT...,hob.3|chr16:82016933-82017204
3,chr10,72334959,72335230,hob.4,chr10:72334959-72335230,8.425374,7.987680,-0.437694,CCCCCATTCGCCGGGCTGCAGAAGGAGGGAGCCTGCTACCAGCAAC...,hob.4|chr10:72334959-72335230
4,chr3,79005150,79005421,hob.5,chr3:79005150-79005421,6.222658,5.418704,-0.803954,TTGGTGACGGTTTCCAGCTTCAAATGAATCAGCTCTCCTTGCCCAG...,hob.5|chr3:79005150-79005421


In [11]:
table.shape

(80892, 10)

##  min read depth filter

In [12]:
## mean read depth filter
if MIN_CONC_FILTER is True:
    table = minReadDepthFilter(table, MIN_CONC) 

## quantile filter
if QUANTILE_FILTER is True:

    table = quantileFilter(table, quantile=0.99)

## joint accessible filter
if JOINT_ACCESSIBLE is False:
    table = jointAccessibleFilter(table)

## apply class label
if CLASS_LABEL is True:
    table = diffAccLabel(table, DIFF_BED)
    print(table.groupby(['ctrl', "US", "US_DIF", "diffAcc"])['#chr'].count())
    
if DIRECTION_FILTER is True:
    table = directionFilter(table)

ctrl       US         US_DIF     diffAcc
2.735509   4.092141   -1.356631  1          1
2.861325   4.300509   -1.439185  1          1
2.921416   4.008265   -1.086850  0          1
2.932169   4.040911   -1.108742  0          1
2.938538   4.264894   -1.326356  1          1
                                           ..
10.519145  10.680225  -0.161080  0          1
10.523657  10.680369  -0.156712  0          1
10.527785  10.750965  -0.223180  0          1
10.549858  10.750119  -0.200261  0          1
10.632526  10.815728  -0.183202  0          1
Name: #chr, Length: 80878, dtype: int64


In [13]:
table

,#chr,start_trim,end_trim,type,seq.id,US,ctrl,US_DIF,seq,coor.type,diffAcc
0,chr20,35771828,35772099,hob.1,chr20:35771828-35772099,8.646043,8.194171,-0.451872,TAGGGCGCGGGCCTGTTTCCCGCGCGTCAGGGTAGTGGGCGTGGCC...,hob.1|chr20:35771828-35772099,1
1,chr3,127075008,127075279,hob.2,chr3:127075008-127075279,7.579306,6.978627,-0.600679,GTGAGCTGGGCTATGCTTGGTTGGTCTGGGCTGAGCTCCACGGTCT...,hob.2|chr3:127075008-127075279,1
2,chr16,82016933,82017204,hob.3,chr16:82016933-82017204,4.529926,3.056418,-1.473507,CTAGAAACATAGCCTTTAGAGACAGAGTGATGGAGATTTGGATCCT...,hob.3|chr16:82016933-82017204,1
3,chr10,72334959,72335230,hob.4,chr10:72334959-72335230,8.425374,7.987680,-0.437694,CCCCCATTCGCCGGGCTGCAGAAGGAGGGAGCCTGCTACCAGCAAC...,hob.4|chr10:72334959-72335230,1
4,chr3,79005150,79005421,hob.5,chr3:79005150-79005421,6.222658,5.418704,-0.803954,TTGGTGACGGTTTCCAGCTTCAAATGAATCAGCTCTCCTTGCCCAG...,hob.5|chr3:79005150-79005421,1
...,...,...,...,...,...,...,...,...,...,...,...
80887,chr1,45550657,45550928,hob.159532,chr1:45550657-45550928,7.216230,7.218726,0.002495,ACGGATGGCATTGTGAATCCGGAGGGCCGACACCAGAAGAACCTGC...,hob.159532|chr1:45550657-45550928,0
80888,chr3,194494080,194494351,hob.159535,chr3:194494080-194494351,6.065404,6.066099,0.000695,actttgctccagggccaaactcttgaccactatAACCAAATGAAGC...,hob.159535|chr3:194494080-194494351,0
80889,chr8,25418233,25418504,hob.159536,chr8:25418233-25418504,4.884313,4.883241,-0.001072,GCAAGTTATCTGAAATTTGTTTTTTTTTTTCCAATGGTCTCATAGC...,hob.159536|chr8:25418233-25418504,0
80890,chr8,53917937,53918208,hob.159537,chr8:53917937-53918208,4.223117,4.214222,-0.008895,ctctagacatacctgttctgggcatttcatgtacaagtattataca...,hob.159537|chr8:53917937-53918208,0


## train on all atac peaks

In [16]:
os.chdir(DATA_PATH)

table = dfShuffle(table)  # shuffle the table before splitting

train, val, test = splitTrainTestVal(table, val_chr_list=[VAL_CHR], test_chr_list=TEST_CHR)

writeFa(test[fa_cols], TEST_FA)
writeFa(train[fa_cols], TRAIN_FA)
writeFa(val[fa_cols], VAL_FA)

In [18]:
if "reg" in PREFIX:
    
    cols = ["coor.type",
        "US",
        "ctrl", 
       # "US_DIF"
       ]
elif "class" in PREFIX:
    cols=["coor.type", "diffAcc"]
    
test[cols].to_csv(TEST_TARGET, sep='\t', index=False)
train[cols].to_csv(TRAIN_TARGET, sep='\t', index=False)
val[cols].to_csv(VAL_TARGET, sep='\t', index=False)

In [19]:
val.shape, train.shape, test.shape

((4011, 11), (73339, 11), (3542, 11))

In [24]:
train.loc[train["diffAcc"]>0] # ctrl - US <0 means US has more activity than control

,#chr,start_trim,end_trim,type,seq.id,US,ctrl,US_DIF,seq,coor.type,diffAcc
148,chr22,26471549,26471820,hob.974,chr22:26471549-26471820,5.845527,5.294266,-0.551261,ATTCTTCATAGCCTCAAGACTGTGCCATCCCAGCCAGCCATTCTAT...,hob.974|chr22:26471549-26471820,1
229,chr17,40486124,40486395,hob.1494,chr17:40486124-40486395,8.916594,8.677527,-0.239067,TGCTAGAGCTTAGCTTCTGGGTTTACTGCTTGGGTTTGTTCTTCTT...,hob.1494|chr17:40486124-40486395,1
276,chr14,69317134,69317405,hob.2060,chr14:69317134-69317405,7.749388,7.454315,-0.295073,ccaggatctccagtctcttgagctggtccagatatagctcctcatg...,hob.2060|chr14:69317134-69317405,1
310,chr15,92882411,92882682,hob.92,chr15:92882411-92882682,9.313575,9.011314,-0.302260,CTCAAGAAAACCAGAAACGCCTACGGACAGTGAATGGCACTAAGGC...,hob.92|chr15:92882411-92882682,1
380,chr19,48325951,48326222,hob.1502,chr19:48325951-48326222,7.404366,7.067680,-0.336686,TGGGAGGCTAACTCCTTCTTGGGCCTCTCTTCCCCCCAGCGCGAAT...,hob.1502|chr19:48325951-48326222,1
...,...,...,...,...,...,...,...,...,...,...,...
80805,chr10,48306416,48306687,hob.336,chr10:48306416-48306687,8.305881,8.012875,-0.293006,GCCGCCACAGGGCCCGCCCCTCCCGCTCCAGGCCACGCCCCTCACC...,hob.336|chr10:48306416-48306687,1
80842,chr19,44870158,44870429,hob.1228,chr19:44870158-44870429,6.380349,5.924097,-0.456252,GCACAAAATCACCAACCAAAACCCCAAAGTGGGTGGAGGCAGAGAC...,hob.1228|chr19:44870158-44870429,1
80859,chr10,48634381,48634652,hob.341,chr10:48634381-48634652,7.744561,7.359510,-0.385051,TGCTGGGTGCAGAGGGCGGTGAGCCTGCACTGTGGTTTGCCCATTT...,hob.341|chr10:48634381-48634652,1
80866,chr5,73448639,73448910,hob.2049,chr5:73448639-73448910,8.334268,8.088882,-0.245387,CTCGGAGCAGCTTTTGCCCCGTCAGCTGAGGAGGGGGTGGCGGGGC...,hob.2049|chr5:73448639-73448910,1


In [25]:
train.loc[train["diffAcc"]>0].shape[0]/train.shape[0]

0.025020793847748128

In [26]:
val.loc[val["diffAcc"]>0].shape[0]/val.shape[0]

0.02144103714784343

In [27]:
test.loc[test["diffAcc"]>0].shape[0]/test.shape[0]

0.018351214003387916

In [30]:
1024/train.shape[0]

0.01396255743874337

In [31]:
64/train.shape[0]

0.0008726598399214606